#1. Project Overview & Business Context

# Customer Visit → Order Funnel & Behavior Analysis

## Business Context
Understanding how customer activity translates into completed orders is critical
for monitoring business performance.

While explicit visit logs are not always available, customer presence and
purchase timestamps can be used as proxies to analyze conversion behavior
and engagement patterns.

This project demonstrates an end-to-end analysis of customer activity and order
data using a public e-commerce dataset, focusing on funnel construction,
customer behavior, and actionable business insights.

#2. Business Question

## Business Questions
1. How many customers in the system convert into actual orders?
2. How does customer behavior differ between single-purchase and repeat customers?
3. How long does it typically take for customers to place repeat orders?
4. What insights can help improve conversion and retention performance?


#3. Load Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

customers = pd.read_csv("olist_customers_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")

customers.head(), orders.head()

(                        customer_id                customer_unique_id  \
 0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
 1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
 2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
 3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
 4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   
 
    customer_zip_code_prefix          customer_city customer_state  
 0                     14409                 franca             SP  
 1                      9790  sao bernardo do campo             SP  
 2                      1151              sao paulo             SP  
 3                      8775        mogi das cruzes             SP  
 4                     13056               campinas             SP  ,
                            order_id                       customer_id  \
 0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10

#4. Data Understanding

## Data Understanding
- Customers: basic customer identifiers and location.
- Orders: order lifecycle timestamps and status.
- Order items: detailed transaction values.

These tables are combined to analyze customer activity, conversion,
and purchasing behavior.


#5. Data Cleaning & Preparation

In [ ]:
orders["order_purchase_timestamp"] = pd.to_datetime(
    orders["order_purchase_timestamp"], errors="coerce"
)

orders = orders.dropna(subset=["order_purchase_timestamp"])
orders = orders.drop_duplicates(subset=["order_id"])

# Aggregate order value
order_value = (
    order_items.groupby("order_id")
    .agg(total_order_value=("price", "sum"))
    .reset_index()
)

orders = orders.merge(order_value, on="order_id", how="left")
orders["total_order_value"] = orders["total_order_value"].fillna(0)

orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,total_order_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,19.90


#6. Data Quality Consideration

## Data Quality Considerations
Before analyzing the funnel, data duplicates and missing timestamps were
addressed to ensure accurate attribution of customer behavior.
Order-level aggregation was performed to avoid double counting transactions.


#7. Customer Level Funnel - Construction

In [ ]:
customer_orders = (
    orders.merge(customers, on="customer_id", how="left")
    .groupby("customer_unique_id")
    .agg(
        first_order=("order_purchase_timestamp", "min"),
        total_orders=("order_id", "nunique"),
        total_revenue=("total_order_value", "sum")
    )
    .reset_index()
)

customer_orders.head()

,customer_unique_id,first_order,total_orders,total_revenue
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,129.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,18.90
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,69.00
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,25.99
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,180.00


#8. Funnel Definition & Assumptions

## Funnel Definition
- Customer exists in the system → considered as a visit proxy.
- First completed order → conversion event.
- Multiple orders → repeat behavior.

This approach reflects common analytical practice when explicit visit
tracking is unavailable.


#9. Funnel Metrics

In [ ]:
total_customers = customers["customer_unique_id"].nunique()
converted_customers = customer_orders.shape[0]
repeat_customers = (customer_orders["total_orders"] > 1).sum()

total_customers, converted_customers, repeat_customers

(96096, 96096, np.int64(2997))

#10. Funnel Findings

## Funnel Findings
- Only a subset of customers in the system proceed to place orders.
- Among converted customers, a smaller group becomes repeat buyers.
- This indicates a clear drop-off between initial conversion and retention.


#11.Customer Behaviour Analysis

In [ ]:
customer_orders["customer_type"] = np.where(
    customer_orders["total_orders"] > 1, "Repeat", "Single"
)

customer_orders["customer_type"].value_counts()


,count
customer_type,
Single,93099
Repeat,2997


In [ ]:
customer_orders.groupby("customer_type")["total_revenue"].mean()

,total_revenue
customer_type,
Repeat,259.867191
Single,137.625772


#12. Behavioural Findings

## Customer Behavior Findings
- Repeat customers generate significantly higher average revenue than
  single-purchase customers.
- Retention appears to be a more effective driver of revenue growth than
  acquiring new one-time buyers alone.


#13. Time-to-repeat Purchase Analysis

In [ ]:
order_times = (
    orders.merge(customers, on="customer_id", how="left")
    .sort_values(["customer_unique_id", "order_purchase_timestamp"])
)

order_times["previous_order_time"] = (
    order_times.groupby("customer_unique_id")["order_purchase_timestamp"].shift(1)
)

order_times["days_since_last_order"] = (
    order_times["order_purchase_timestamp"] - order_times["previous_order_time"]
).dt.days

order_times.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,total_order_value,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,previous_order_time,days_since_last_order
52798,e22acc9c116caa3f2b7121bbb380d08e,fadbb3709178fc513abc1b2670aa1ad2,delivered,2018-05-10 10:56:27,2018-05-10 11:11:18,2018-05-12 08:18:00,2018-05-16 20:48:37,2018-05-21 00:00:00,129.90,0000366f3b9a7992bf8c76cfdf3221e2,7787,cajamar,SP,NaT,NaN
73889,3594e05a005ac4d06a72673270ef9ec9,4cb282e167ae9234755102258dd52ee8,delivered,2018-05-07 11:11:27,2018-05-07 18:25:44,2018-05-09 12:18:00,2018-05-10 18:02:42,2018-05-15 00:00:00,18.90,0000b849f77a49e4a4ce2b2a4ca5be3f,6053,osasco,SP,NaT,NaN
26460,b33ec3b699337181488304f362a6b734,9b3932a6253894a02c1df9d19004239f,delivered,2017-03-10 21:05:03,2017-03-10 21:05:03,2017-03-13 12:58:30,2017-04-05 14:38:47,2017-04-07 00:00:00,69.00,0000f46a3911fa3c0805444483337064,88115,sao jose,SC,NaT,NaN
98493,41272756ecddd9a9ed0180413cc22fb6,914991f0c02ef0843c0e7010c819d642,delivered,2017-10-12 20:29:41,2017-10-12 20:49:17,2017-10-13 20:08:19,2017-11-01 21:23:05,2017-11-13 00:00:00,25.99,0000f6ccb0745a6a4b88665a16c9f078,66812,belem,PA,NaT,NaN
41564,d957021f1127559cd947b62533f484f7,47227568b10f5f58a524a75507e6992c,delivered,2017-11-14 19:45:42,2017-11-14 20:06:52,2017-11-16 19:52:10,2017-11-27 23:08:56,2017-12-05 00:00:00,180.00,0004aac84e0df4da2b147fca70cf8255,18040,sorocaba,SP,NaT,NaN


#14. Time to Order Findings

## Time-to-Order Findings
Customers who place repeat orders within shorter time intervals are more likely
to develop long-term purchasing relationships.
Early engagement plays a critical role in retention.


#15. Final Insight & Business Recommendation

## Final Insights & Business Recommendations

Customer conversion and retention are uneven across the funnel, with significant
drop-off occurring after the first purchase.
Repeat customers contribute substantially higher revenue, highlighting retention
as a primary driver of business value.

To improve overall performance, businesses should prioritize strategies that
encourage early repeat purchases, monitor customer conversion funnels regularly,
and distinguish between one-time and long-term customer behavior.

Clear visibility into customer activity and purchase patterns enables more
effective, data-driven decisions around marketing focus, retention initiatives,
and performance monitoring.
